**Programmer:** Marco Gutierrez

**Note:** Don't run the whole code in once. Twitter imposes a constrain of 300 requests approx. per 15 mins, so take that into account when running this script

# Importing packages

In [3]:
import pandas as pd
import twitter
from selenium import webdriver
import nltk
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [2]:
# importing our congress dataframe
congress_base = pd.read_excel("datos_congress.xlsx")

In [3]:
congress_base.iloc[:6]

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe
5,Aliaga Pajares Guillermo Alejandro Antonio,SOMOS PERÚ,galiaga@congreso.gob.pe


## Getting first names

In [50]:
def get_first_names(row):
    """Function to get first names
    
    Input: row string value with full name
    Output: first name
    """
    splitted_names = row.split(" ")
    length = len(splitted_names)

    if length>=5:
        first_names = splitted_names[-1] + " " + splitted_names[-2] + " " + splitted_names[-3]
    
    elif length==4:
        first_names = splitted_names[-1] + " " + splitted_names[-2]
        
    elif length==3 or length==2:
        first_names = splitted_names[-1]
            
    return first_names

In [51]:
congress_base["Nombres"] = congress_base["APELLIDOS Y NOMBRES"].apply(lambda row: get_first_names(row))

In [6]:
congress_base["Primer Nombre"] = congress_base["APELLIDOS Y NOMBRES"].apply(lambda row: get_first_names(row).split(" ")[0])

In [58]:
def get_second_name(row):
    if len(row.split(" "))>1:
        return row.split(" ")[1]
    else:
        return "No encontrado"

In [59]:
congress_base["Segundo Nombre"] = congress_base["Nombres"].apply(lambda row: get_second_name(row))

In [60]:
congress_base.head()

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL,Nombres,Primer Nombre,Apellidos,Primer Apellido,Nombre Completo,Nombre Completo (F),Nombre Completo (FFL),Usuario de Twitter (1st FullN),Usuario de Twitter (1st FirstN),Usuario de Twitter (1st FirstN FullL),description (1st FullN),Segundo Nombre
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe,Geovanni Eduardo,Geovanni,Acate Coronel,Acate,Geovanni Eduardo Acate Coronel,Geovanni Acate,Geovanni Acate Coronel,No encontrado,geovanniacate,No encontrado,Not found,Eduardo
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe,Humberto,Humberto,Acuña Peralta,Acuña,Humberto Acuña Peralta,Humberto Acuña,Humberto Acuña Peralta,HumbertoAcunaP,HumbertoAcunaP,HumbertoAcunaP,Congresista de la República por @Peru_APP\nExg...,No encontrado
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe,Manuel,Manuel,Aguilar Zamora,Aguilar,Manuel Aguilar Zamora,Manuel Aguilar,Manuel Aguilar Zamora,JosManu41917702,Manuelsv,JosManu41917702,,No encontrado
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe,Arnold Edgar,Arnold,Alarcón Tejada,Alarcón,Arnold Edgar Alarcón Tejada,Arnold Alarcón,Arnold Alarcón Tejada,No encontrado,LobitoDenis,No encontrado,Not found,Edgar
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe,Norma Hirma,Norma,Alencastre Miranda,Alencastre,Norma Hirma Alencastre Miranda,Norma Alencastre,Norma Alencastre Miranda,No encontrado,NormaAlencastre,No encontrado,Not found,Hirma


## Getting last names

In [8]:
def get_last_names(row):
    """Function to get last names
    
    Input: row string value with full name
    Output: last name
    """
    splitted_names = row.split(" ")
    length = len(splitted_names)

    if length<=4:
        last_names = splitted_names[0] + " " + splitted_names[1]
    elif length>=5:
        last_names = splitted_names[0] + " " + splitted_names[1] + " " + splitted_names[2] 
    
    return last_names

In [9]:
congress_base["Apellidos"] = congress_base["APELLIDOS Y NOMBRES"].apply(lambda row: get_last_names(row))

### Getting first last name

In [10]:
def get_first_last_name(row):
    """Function to get last names
    
    Input: row string value with last name
    Output: first last name"""
    
    splitted_last_names = row.split(" ")
    length = len(splitted_last_names)

    if length==2:
        first_last_name = splitted_last_names[0]
    elif length==3:
        first_last_name = splitted_last_names[0] + " " + splitted_last_names[1]
    
    return first_last_name

In [11]:
congress_base["Primer Apellido"] = congress_base["Apellidos"].apply(lambda row: get_first_last_name(row))

In [12]:
congress_base.head()

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL,Nombres,Primer Nombre,Apellidos,Primer Apellido
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe,Geovanni Eduardo,Geovanni,Acate Coronel,Acate
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe,Humberto,Humberto,Acuña Peralta,Acuña
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe,Manuel,Manuel,Aguilar Zamora,Aguilar
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe,Arnold Edgar,Arnold,Alarcón Tejada,Alarcón
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe,Norma Hirma,Norma,Alencastre Miranda,Alencastre


## Creating Full Name Ordered

In [13]:
congress_base["Nombre Completo"] = congress_base["Nombres"] + " " + congress_base["Apellidos"]

In [14]:
congress_base.head()

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL,Nombres,Primer Nombre,Apellidos,Primer Apellido,Nombre Completo
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe,Geovanni Eduardo,Geovanni,Acate Coronel,Acate,Geovanni Eduardo Acate Coronel
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe,Humberto,Humberto,Acuña Peralta,Acuña,Humberto Acuña Peralta
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe,Manuel,Manuel,Aguilar Zamora,Aguilar,Manuel Aguilar Zamora
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe,Arnold Edgar,Arnold,Alarcón Tejada,Alarcón,Arnold Edgar Alarcón Tejada
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe,Norma Hirma,Norma,Alencastre Miranda,Alencastre,Norma Hirma Alencastre Miranda


## Creating Full Name (Only firsts F and L Names)

In [15]:
congress_base["Nombre Completo (F)"] = congress_base["Primer Nombre"] + " " + congress_base["Primer Apellido"]

In [16]:
congress_base.head()

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL,Nombres,Primer Nombre,Apellidos,Primer Apellido,Nombre Completo,Nombre Completo (F)
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe,Geovanni Eduardo,Geovanni,Acate Coronel,Acate,Geovanni Eduardo Acate Coronel,Geovanni Acate
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe,Humberto,Humberto,Acuña Peralta,Acuña,Humberto Acuña Peralta,Humberto Acuña
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe,Manuel,Manuel,Aguilar Zamora,Aguilar,Manuel Aguilar Zamora,Manuel Aguilar
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe,Arnold Edgar,Arnold,Alarcón Tejada,Alarcón,Arnold Edgar Alarcón Tejada,Arnold Alarcón
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe,Norma Hirma,Norma,Alencastre Miranda,Alencastre,Norma Hirma Alencastre Miranda,Norma Alencastre


## Creating Full Name (First F and Full L Names)

In [17]:
congress_base["Nombre Completo (FFL)"] = congress_base["Primer Nombre"] + " " + congress_base["Apellidos"]

In [37]:
congress_base.head()

APELLIDOS Y NOMBRES                      Bajonero Olivas Wilmer Solis
GRUPO PARLAMENTARIO                                    ACCION POPULAR
E-MAIL                                      wbajonero@congreso.gob.pe
Nombres                                                  Solis Wilmer
Primer Nombre                                                   Solis
Apellidos                                             Bajonero Olivas
Primer Apellido                                              Bajonero
Nombre Completo                          Solis Wilmer Bajonero Olivas
Nombre Completo (F)                                    Solis Bajonero
Nombre Completo (FFL)                           Solis Bajonero Olivas
Usuario de Twitter (1st FullN)                          No encontrado
Usuario de Twitter (1st FirstN)                       Hugo_Bajonero01
Usuario de Twitter (1st FirstN FullL)                   No encontrado
description (1st FullN)                                     Not found
Name: 14, dtype: obj

# Creating Full Name (2nd F and 1st L Name)

In [ ]:
congress_base["Nombre Completo (2F1L)"] = congress_base["Primer Nombre"] + " " + congress_base["Apellidos"]

# Calling our twitter API

In [5]:
api = twitter.Api(consumer_key=os.environ.get('consumer_key_twt'),
                  consumer_secret=os.environ.get('consumer_secret_twt'),
                  access_token_key=os.environ.get('access_token_key_twt'),
                  access_token_secret=os.environ.get('access_token_secret_twt'))

# Scraping user names

Because the first result may not be the actual account, we'll compare the first results when scraping with their full names and only with the 'first' first and last name

In [20]:
def get_user_name(real_name, max_nmb_users, desired_user):
    """
    Gets the user name of a person
    
    Input: real name, max number of users to be looked for, number of desired result (from 1 to max_nmb_users)
    Output: twitter accounts 
    """
    users = api.GetUsersSearch(term=real_name, count=max_nmb_users)
    nmb_users_found = len(users)
    
    if nmb_users_found==0:
        return "No encontrado"
    elif nmb_users_found>0:
        if desired_user<=nmb_users_found and desired_user>0: # if desired user is in range
            return users[desired_user-1].screen_name
        
        elif desired_user>nmb_users_found: #if out of range
            return "Fuera de rango"
        
        else:
            print("Invalid value inputed. Input a positive integer (greater than 0)")

### Twitter Username (Full Name)

In [21]:
congress_base['Usuario de Twitter (1st FullN)'] = congress_base["Nombre Completo"].apply(lambda row: get_user_name(row, 3, 1))

### Twitter Username (First Names)

In [63]:
congress_base['Usuario de Twitter (1st FirstN)'] = congress_base["Nombre Completo (F)"].apply(lambda row: get_user_name(row, 3, 1))

### Twitter Username (First + Full Lastnames)

In [70]:
congress_base['Usuario de Twitter (1st FirstN FullL)'] = congress_base["Nombre Completo (FFL)"].apply(lambda row: get_user_name(row, 3, 1))

### Twitter Username (Second First + First Lastnames)

In [89]:
congress_base['Nombre Completo (SFFL)'] = congress_base['Segundo Nombre'] + " " + congress_base['Primer Apellido']

In [90]:
congress_base['Usuario de Twitter (2nd FirstN FirstL)'] = congress_base["Nombre Completo (SFFL)"].apply(lambda row: get_user_name(row, 3, 1))

## Twitter Usernames (Second result)

### Twitter Username (Full Name)

In [166]:
congress_base['Usuario de Twitter (1st FullN) 2'] = congress_base["Nombre Completo"].apply(lambda row: get_user_name(row, 2, 2))

### Twitter Username (First Names)

In [167]:
congress_base['Usuario de Twitter (1st FirstN) 2'] = congress_base["Nombre Completo (F)"].apply(lambda row: get_user_name(row, 2, 2))

### Twitter Username (First + Full Lastnames)

In [168]:
congress_base['Usuario de Twitter (1st FirstN FullL) 2'] = congress_base["Nombre Completo (FFL)"].apply(lambda row: get_user_name(row, 2, 2))

### Twitter Username (Second First + First Lastnames)

In [170]:
congress_base['Usuario de Twitter (2nd FirstN FirstL) 2'] = congress_base["Nombre Completo (SFFL)"].apply(lambda row: get_user_name(row, 2, 2))

In [191]:
congress_base.iloc[33:34]['description (2nd FirstN FirstL)']

33    
Name: description (2nd FirstN FirstL), dtype: object

# List of names

In [192]:
list_of_names = ["Nombre Completo", "Nombre Completo (F)", "Nombre Completo (FFL)", "Nombre Completo (SFFL)"]

# List of scrapes done

In [210]:
list_of_scrapes = ["(1st FullN)", "(1st FirstN)", "(1st FirstN FullL)", "(2nd FirstN FirstL)", "(1st FullN) 2", "(1st FirstN) 2", "(1st FirstN FullL) 2", "(2nd FirstN FirstL) 2"]

# Combined list

In [214]:
first_combined_list = list(zip(list_of_names, list_of_scrapes)) # first users
second_combined_list = list(zip(list_of_names, list_of_scrapes[int(len(list_of_scrapes)/2):]))# second users

# Extracting descriptions of each account

In [79]:
def getting_descriptions(user_name):
    
    if user_name != "No encontrado" and user_name != "Fuera de rango":
        user = api.GetUsersSearch(term=user_name)
        if len(user)>0:
            return user[0].description
        else:
            return "Usuario Abandonado"
    else:
        return "No encontrado"
        

## Scraping first user's descriptions

In [ ]:
for item in first_combined_list:
    congress_base[f"description {item[1]}"] = congress_base[f"{item[0]}"].apply(lambda row: getting_descriptions(row))

## Scraping second user's descriptions

In [175]:
for item in second_combined_list:
    congress_base[f"description {item[1]} 2"] = congress_base[f"{item[0]}"].apply(lambda row: getting_descriptions(row))

(1st FullN) 2
finished (1st FullN) 2
(1st FirstN) 2
finished (1st FirstN) 2
(1st FirstN FullL) 2
finished (1st FirstN FullL) 2
(2nd FirstN FirstL) 2
finished (2nd FirstN FirstL) 2


In [176]:
congress_base.head()

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL,Nombres,Primer Nombre,Apellidos,Primer Apellido,Nombre Completo,Nombre Completo (F),Nombre Completo (FFL),...,score (2nd FirstN FirstL),Best match,Usuario de Twitter (1st FullN) 2,Usuario de Twitter (1st FirstN) 2,Usuario de Twitter (1st FirstN FullL) 2,Usuario de Twitter (2nd FirstN FirstL) 2,description (1st FullN) 2,description (1st FirstN) 2,description (1st FirstN FullL) 2,description (2nd FirstN FirstL) 2
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe,Geovanni Eduardo,Geovanni,Acate Coronel,Acate,Geovanni Eduardo Acate Coronel,Geovanni Acate,Geovanni Acate Coronel,...,0.0,geovanniacate,No encontrado,Fuera de rango,No encontrado,No encontrado,No encontrado,Programa de radio Inicio: 19.02.2015 Finalizó:...,No encontrado,No encontrado
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe,Humberto,Humberto,Acuña Peralta,Acuña,Humberto Acuña Peralta,Humberto Acuña,Humberto Acuña Peralta,...,0.0,HumbertoAcunaP,humberto_ap,huacuna,humberto_ap,No encontrado,Gobernador regional de Lambayeque,,Gobernador regional de Lambayeque,No encontrado
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe,Manuel,Manuel,Aguilar Zamora,Aguilar,Manuel Aguilar Zamora,Manuel Aguilar,Manuel Aguilar Zamora,...,0.0,No match,ManuelAguilarz1,cornellius,ManuelAguilarz1,No encontrado,,Coping with the midlife crisis i altres. Catal...,,No encontrado
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe,Arnold Edgar,Arnold,Alarcón Tejada,Alarcón,Arnold Edgar Alarcón Tejada,Arnold Alarcón,Arnold Alarcón Tejada,...,0.0,No match,No encontrado,ArnoldAlarcon1,No encontrado,EdgarAlarconT,No encontrado,gamer and basketball player,No encontrado,Congresista de la República (2020-2021). Arequ...
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe,Norma Hirma,Norma,Alencastre Miranda,Alencastre,Norma Hirma Alencastre Miranda,Norma Alencastre,Norma Alencastre Miranda,...,0.0,NormaAlencastre,No encontrado,noalro_5,No encontrado,No encontrado,No encontrado,Usuario Abandonado,No encontrado,No encontrado


# Score for detecting Congressmen

In [222]:
congress_words = ["congreso", "congresista", "politica", "política", "soci", "sociedad", "social", 
                  "bancada", "Perú", "republica", "república", "democracia"]

ngrams_congress = []
for word in congress_words:
    ngrams = nltk.ngrams(word, 4)
    for gram in ngrams:
        ngrams_congress.append(''.join(gram))

In [223]:
def congresscore(description, ngrams):
    score = 0
    for gram in ngrams:
        if gram in description:
            score +=1
    return score/len(ngrams_congress)

In [224]:
for scrape in list_of_scrapes:
    congress_base[f"score {scrape}"] = congress_base[f"description {scrape}"].apply(lambda row: congresscore(row, ngrams_congress))

In [227]:
congress_base.head()

,APELLIDOS Y NOMBRES,GRUPO PARLAMENTARIO,E-MAIL,Nombres,Primer Nombre,Apellidos,Primer Apellido,Nombre Completo,Nombre Completo (F),Nombre Completo (FFL),...,Usuario de Twitter (1st FirstN FullL) 2,Usuario de Twitter (2nd FirstN FirstL) 2,description (1st FullN) 2,description (1st FirstN) 2,description (1st FirstN FullL) 2,description (2nd FirstN FirstL) 2,score (1st FullN) 2,score (1st FirstN) 2,score (1st FirstN FullL) 2,score (2nd FirstN FirstL) 2
0,Acate Coronel Eduardo Geovanni,ALIANZA PARA EL PROGRESO,eacate@congreso.gob.pe,Geovanni Eduardo,Geovanni,Acate Coronel,Acate,Geovanni Eduardo Acate Coronel,Geovanni Acate,Geovanni Acate Coronel,...,No encontrado,No encontrado,No encontrado,Programa de radio Inicio: 19.02.2015 Finalizó:...,No encontrado,No encontrado,0.0,0.0,0.0,0.000000
1,Acuña Peralta Humberto,ALIANZA PARA EL PROGRESO,hacuna@congreso.gob.pe,Humberto,Humberto,Acuña Peralta,Acuña,Humberto Acuña Peralta,Humberto Acuña,Humberto Acuña Peralta,...,humberto_ap,No encontrado,Gobernador regional de Lambayeque,,Gobernador regional de Lambayeque,No encontrado,0.0,0.0,0.0,0.000000
2,Aguilar Zamora Manuel,ACCION POPULAR,maguilarz@congreso.gob.pe,Manuel,Manuel,Aguilar Zamora,Aguilar,Manuel Aguilar Zamora,Manuel Aguilar,Manuel Aguilar Zamora,...,ManuelAguilarz1,No encontrado,,Coping with the midlife crisis i altres. Catal...,,No encontrado,0.0,0.0,0.0,0.000000
3,Alarcón Tejada Edgar Arnold,UNIÓN POR EL PERÚ,ealarcont@congreso.gob.pe,Arnold Edgar,Arnold,Alarcón Tejada,Alarcón,Arnold Edgar Alarcón Tejada,Arnold Alarcón,Arnold Alarcón Tejada,...,No encontrado,EdgarAlarconT,No encontrado,gamer and basketball player,No encontrado,Congresista de la República (2020-2021). Arequ...,0.0,0.0,0.0,0.303571
4,Alencastre Miranda Hirma Norma,SOMOS PERÚ,halencastre@congreso.gob.pe,Norma Hirma,Norma,Alencastre Miranda,Alencastre,Norma Hirma Alencastre Miranda,Norma Alencastre,Norma Alencastre Miranda,...,No encontrado,No encontrado,No encontrado,Usuario Abandonado,No encontrado,No encontrado,0.0,0.0,0.0,0.000000


# Getting the user with the highest score among the others

In [228]:
def getting_best_match(row):
    best_match = "No match"
    benchmark = 0
    
    for scrape in list_of_scrapes:
        #print(row[f"score {scrape}"])
        if row[f"score {scrape}"]>benchmark:
            best_match = row[f"Usuario de Twitter {scrape}"]
            benchmark = row[f"score {scrape}"]
    return best_match

In [229]:
congress_base["Best match"] = congress_base.apply(lambda row: getting_best_match(row), axis=1)

In [230]:
congress_base.iloc[33]

APELLIDOS Y NOMBRES                                                   Chehade Moya Omar Karim
GRUPO PARLAMENTARIO                                                  ALIANZA PARA EL PROGRESO
E-MAIL                                                               ochehade@congreso.gob.pe
Nombres                                                                            Karim Omar
Primer Nombre                                                                           Karim
Apellidos                                                                        Chehade Moya
Primer Apellido                                                                       Chehade
Nombre Completo                                                       Karim Omar Chehade Moya
Nombre Completo (F)                                                             Karim Chehade
Nombre Completo (FFL)                                                      Karim Chehade Moya
Usuario de Twitter (1st FullN)                              

# Generating output database

In [231]:
output_users = congress_base[["Nombre Completo", "GRUPO PARLAMENTARIO", "Best match"]]

In [232]:
output_users.to_excel("congress twitter best matches.xlsx")

D:\Anaconda5.2.0\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.3' currently installed).
  warnings.warn(msg, UserWarning)
